In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
import h5py
import tqdm
import zipfile
from tqdm.notebook import tqdm_notebook
from progress.bar import IncrementalBar
import random
import pandas as pd
import time
import os
import math
from joblib import Parallel, delayed
# tf.config.run_functions_eagerly(True)
%matplotlib inline

In [2]:
param_names=['signal','pl_fr','real_wf-pl_fr','mask']

In [3]:
def norming(data,param_names=param_names,log=True):
    
    # ADD LOGARIFM FOR SIGNAL
    norm_params={}
    for i in range(data.shape[-1]):
        if (i==0 and log):
            data[:,:,:,i]=tf.math.log(data[:,:,:,i]+1)
        max_c=data[:,:,:,i].max()
        min_c=data[:,:,:,i].min()
        mean_c=data[:,:,:,i].mean()
        std_c=data[:,:,:,i].std()
        print(max_c,min_c)
        data[:,:,:,i]=(data[:,:,:,i]-min_c)/(max_c-min_c)

        norm_params[param_names[i]]=np.array([max_c,min_c])
    return norm_params
def renormin(data_not_change,norm_params,param_names=param_names,log=True):
    data=np.copy(data_not_change)
    for i in range(data.shape[-1]):
        max_c, min_c=norm_params[param_names[i]]
        data[:,:,:,i]=data[:,:,:,i]*(max_c-min_c) + min_c
        if (i==0 and log):
            data[:,:,:,i]=tf.math.exp(data[:,:,:,i])-1
        elif (i==1 or i==2):
            data[:,:,:,i]=data[:,:,:,i]*1e6
    return data

In [204]:
path='../../mc_hadrons_qgs34_0010.h5'
data_name='pr-q4-9yr'
data=np.zeros((1,6,6,4))
with h5py.File(path,'r') as f:
    data=np.concatenate((data,f[data_name]['dt'][:100,:,:,3:7]))
    gp=f[data_name]['gp'].value
    ev_params=f[data_name]['ev_params'].value
data=data[1:]
data[:,:,:,1]=data[:,:,:,1]*1e6
data[:,:,:,2]=data[:,:,:,2]*1e6
print('len data: ',len(data))

len data:  100


/home/rfit/.conda/envs/robert_venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/home/rfit/.conda/envs/robert_venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  import sys


# Constant

In [205]:

pi = tf.constant(math.pi)
UNIT=1
dist=tf.constant(1.2,dtype=tf.float32)# min dist between 2 detectors in km
c=tf.constant(299792.458,dtype=tf.float32)# # km\s
NSEC= 1e6/c# in rubsov's code is a time for 1.2 km 1.2/c*1e9
R_L=30e-3
LINSLEY_r0=0.025
Det_Area=30
tf_type=tf.float32
s_min = tf.constant([[0.3]])
s_max = tf.constant([[1.8]])
t_err_res=c/1e6
t0_err=30

In [246]:
class Reconstruction():
    #constants
    pi = tf.constant(math.pi)
    UNIT=1
    dist=tf.constant(1.2,dtype=tf.float32)# min dist between 2 detectors in km
    c=tf.constant(299792.458,dtype=tf.float32)# # km\s
    NSEC= 1e6/c# in rubsov's code is a time for 1.2 km 1.2/c*1e9
    R_L=30e-3
    LINSLEY_r0=0.025
    Det_Area=3
    s_min = tf.constant([[0.3]])
    s_max = tf.constant([[1.8]])
    t_err_res=c/1e6
    t0_err=30
    
    def __init__(self,data,tf_type):
        # all have shape (batch,6,6,1)
        self.tf_type = tf_type
        self.data=tf.cast(data,self.tf_type)
        self.mask = tf.expand_dims(data[:,:,:,3],-1)
        self.signal = tf.expand_dims(data[:,:,:,0],-1)*self.mask
        self.real_time = tf.expand_dims((data[:,:,:,1]+data[:,:,:,2]),-1)*self.mask
        self.signal = tf.cast(self.signal,self.tf_type)
        self.real_time = tf.cast(self.real_time,self.tf_type)
        self.mask = tf.cast(self.mask,self.tf_type)
        self.batch = data.shape[0]
        self.t0=None
        self.theta=None
        self.detertors_coord=None
        self.core=None
        self.t_place=None
#     @classmethod
    def detectors_init(self):
        x = tf.cast(tf.repeat(tf.expand_dims(tf.range(0,6),0),6,axis=0),self.tf_type)
        y = tf.cast(tf.repeat(tf.expand_dims(tf.range(0,6),1),6,axis=1),self.tf_type)
        x = tf.expand_dims(x,-1)
        y = tf.expand_dims(y,-1)
        # x.shape = (6,6,1)
        detectors = tf.concat([x,y],axis=-1) * dist
        
        #detectors.shape = (batch,6,6,2) in km
        detectors = tf.repeat(tf.expand_dims(detectors,0),self.batch,0)
        self.detertors_coord=detectors
        return detectors
#     @classmethod
    def core_(self):
        detectors = self.detectors_init() # shape (batch,6,6,2)
        sum_signal=tf.reduce_sum(self.signal,axis=(1,2)) # shape (batch ,1)
        core=tf.reduce_sum(detectors * self.signal, axis=(1,2))/sum_signal
        #update self
        self.core = core
        return core
    @staticmethod
    def expand_dims(vec):
        return tf.expand_dims(tf.expand_dims(vec,-1),-1)
    def create_matrix(self,x,y,t,mask):
        a11=self.expand_dims(tf.reduce_sum(x*x,axis=1))
        a12=self.expand_dims(tf.reduce_sum(x*y,axis=1))
        a13=self.expand_dims(tf.reduce_sum(x,axis=1))
        a22=self.expand_dims(tf.reduce_sum(y*y,axis=1))
        a23=self.expand_dims(tf.reduce_sum(y,axis=1))
        a33=self.expand_dims(tf.reduce_sum(mask,axis=1))
        a1=tf.concat([a11,a12,a13],axis=2)
        a2=tf.concat([a12,a22,a23],axis=2)
        a3=tf.concat([a13,a23,a33],axis=2)
        A=tf.concat([a1,a2,a3],axis=1)

        b1=self.expand_dims(tf.reduce_sum(x*t,axis=1))
        b2=self.expand_dims(tf.reduce_sum(y*t,axis=1))
        b3=self.expand_dims(tf.reduce_sum(t,axis=1))
        b=tf.concat([b1,b2,b3],axis=1)
        return A,b
    def place_sol(self):
        real_time=self.real_time
        mask=self.mask
        detectors = self.detectors_init() * mask
        x = tf.reshape(detectors[:,:,:,0],(-1,36))
        y = tf.reshape(detectors[:,:,:,1],(-1,36))
        t = tf.reshape(real_time,(-1,36))
        mask = tf.reshape(mask,(-1,36))
        A,b=self.create_matrix(x,y,t,mask)
        return tf.linalg.solve(A,b)
#     @classmethod
    def place_params(self):
        sol = self.place_sol()
        #t_0=b+(r_cor;n)
        b=sol[:,2:,0]
        n=sol[:,:2,0]
        core = self.core_()
        mul=n[:,0]*core[:,0]+n[:,1]*core[:,1]
#         t0 = tf.cast(tf.expand_dims(b,1)+tf.expand_dims(mul,1),dtype=self.tf_type)
        t0=b
        a_x=n[:,0]
        a_y=n[:,1]
        a_z=tf.math.pow(1e6/c,2) - (tf.math.pow(a_x,2) + tf.math.pow(a_y,2))
        a_z=tf.where(a_z>0,tf.math.sqrt(a_z),0)
        cos_theta=a_z*(c/1e6)
        theta=tf.math.acos(cos_theta)
        tg_phi=a_y/(-a_x)
        atan=tf.math.atan(tg_phi)
        phi=tf.where(a_y>0,atan+pi,atan-pi)
        phi=tf.where(a_x<0,atan,phi)
        phi = tf.math.atan2( -a_x, -a_y )
        # update self
        self.phi = phi
        self.theta = theta
        self.t0=t0
        return t0,tf.expand_dims(theta,-1),tf.expand_dims(phi,-1)
    def detectors_core(self):
        detectors=self.detertors_coord
        core = self.core
        detectors_c=detectors-tf.reshape(core,(-1,1,1,2))
        return detectors_c
#     @classmethod
    def place_reconstruction(self):
        t0,theta,phi = self.place_params()
        detectors = self.detectors_core()
        t0=self.expand_dims(t0) # shape (batch,1,1,1)
        theta=self.expand_dims(theta)
        phi = self.expand_dims(phi)
        n=tf.concat([tf.math.cos(phi)*tf.math.sin(theta),tf.math.sin(phi)*tf.math.sin(theta)],axis=-1)*(1e6/c)
        t_place = t0  - detectors[:,:,:,0:1]*n[:,:,:,0:1] - detectors[:,:,:,1:2]*n[:,:,:,1:2]
        self.t_place = t_place*self.mask
        return self.t_place

In [247]:
rec.t0

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-1.0818663],
       [-9.411484 ],
       [12.180855 ],
       [ 3.3125648],
       [-2.0338748],
       [-5.917012 ],
       [-7.2979712],
       [ 7.119654 ],
       [ 2.646707 ],
       [ 4.3824105]], dtype=float32)>

In [248]:
rec = Reconstruction(data[:10],tf.float32)
det=rec.detectors_init()
core=rec.core_()
sol = rec.place_sol() 
# sol

In [249]:
tf.math.abs(rec.place_reconstruction()[:10,:,:,0]-data[:10,:,:,1])

<tf.Tensor: shape=(10, 6, 6), dtype=float32, numpy=
array([[[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       ,  5.5269146,  9.538604 ,
          0.       ],
        [ 0.       ,  0.       ,  2.453255 ,  1.614434 ,  5.677123 ,
          0.       ],
        [ 0.       ,  0.       ,  0.       ,  2.008047 ,  0.       ,
          0.       ],
        [ 0.       ,  0.       , 10.087217 ,  0.       ,  0.       ,
          0.       ]],

       [[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , 10.372497 ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  8.894294 ,  9.402746 , 10.612197 ,
          0.       ],
        [ 0.       ,  0.       ,  7.

In [243]:
data[0,:,:,1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -3.59299997, -5.57600015,
         0.        ],
       [ 0.        ,  0.        ,  0.325     , -1.71399995, -3.74799993,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -0.125     ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  3.89200022,  0.        ,  0.        ,
         0.        ]])

In [251]:
ev_params[:10,2:4] 

array([[ 42.4385, 131.57  ],
       [ 40.6731, 245.529 ],
       [ 38.2434,  24.3915],
       [ 38.1833,  87.2113],
       [ 41.7928, 136.063 ],
       [ 46.2211, 298.839 ],
       [ 31.3009, 200.646 ],
       [ 32.9256,  18.2795],
       [ 32.653 , 113.383 ],
       [ 30.5313,  80.6433]], dtype=float32)

In [250]:
tf.where(rec.phi*180/3.14>0,rec.phi*180/3.14,(rec.phi+3.14*2)*180/3.14)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([135.00084 , 239.4015  ,  28.259281, 113.2955  , 139.18312 ,
       289.5204  , 205.41646 ,  20.930283, 114.04648 ,  62.362896],
      dtype=float32)>

In [198]:
t0

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ -1.0818156 ,  -9.411423  ,  12.180873  ,   3.3125598 ,
        -2.033852  ,  -5.9170237 ,  -7.2979884 ,   7.119641  ,
         2.6467204 ,   4.3824077 ,   6.763143  ,   4.004331  ,
        -8.332845  ,   5.4817886 ,   9.464282  ,  -1.446     ,
         9.62021   ,   8.648997  ,  -1.9128022 ,  -7.716711  ,
         4.4970164 ,   3.4287083 ,   2.3299828 ,  -1.0217546 ,
        -1.8223338 ,  -2.5787718 ,  -1.0533029 ,   7.211702  ,
         2.7141802 ,  -8.1832905 ,   4.0757537 ,  -3.485143  ,
         0.9643655 ,   8.388334  ,  -9.509887  ,   6.1425695 ,
        -8.855094  ,  10.695374  ,  -2.2949998 ,  -5.736542  ,
        -6.3683243 ,   3.6549046 ,  -0.33027342,   4.697004  ,
        -0.87361073,   1.6940124 ,  -1.8363557 ,   0.7218607 ,
        -5.4357314 ,  -5.8060293 , -10.523151  ,   6.1079645 ,
        -5.3647585 ,   7.897768  ,  -5.5457034 ,  -3.2928867 ,
        10.168189  ,   8.107447  ,   7.643935  ,  -1.6212804 ,
        

In [185]:
rec.theta*180/3.14

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([45.879997, 35.77847 , 48.284664, 30.336348, 38.42599 , 26.98432 ,
       34.245453, 21.895771, 23.444355, 13.708007], dtype=float32)>

In [187]:
theta*180/3.14

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([45.880013, 35.778294, 48.284744, 30.33634 , 38.425953, 26.98434 ,
       34.24549 , 21.895685, 23.444366, 13.708007, 19.963308, 36.211403,
       34.549713, 26.356588, 24.304092, 20.291754, 31.404182, 35.0867  ,
       40.317444, 39.38651 , 33.438232, 29.687504, 34.592064, 15.464334,
       16.845507, 30.92248 , 25.82884 , 25.760952, 15.69866 , 28.46382 ,
       49.008533, 32.472427, 16.613081, 34.11308 , 38.067448, 26.122942,
       38.344456, 36.396553, 30.778479, 28.356493, 26.999262, 42.776543,
       24.35919 , 15.340359, 10.401696, 29.866833, 39.07704 , 14.71895 ,
       27.42335 , 31.768406, 90.04565 , 90.04565 , 18.409817, 23.17837 ,
       45.431725, 26.252207, 34.34565 , 35.536694, 25.105764, 28.209288,
       36.617958, 33.733616,  8.894859, 31.353144], dtype=float32)>

In [85]:
data[0,:,:,1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -3.59299997, -5.57600015,
         0.        ],
       [ 0.        ,  0.        ,  0.325     , -1.71399995, -3.74799993,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -0.125     ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  3.89200022,  0.        ,  0.        ,
         0.        ]])

In [197]:
num = 64
h5f = '../../mc_hadrons_qgs34_0010.h5'
with h5py.File(h5f,'r') as hf:
    times_flat = hf['pr-q4-9yr/dt'][:num,:,:,4:5]*1e6 # in mks
    times_diff = hf['pr-q4-9yr/dt'][:num,:,:,5:6]*1e6 # in mks
    mask = hf['pr-q4-9yr/dt'][:num,:,:,6:7]
    real_coords = hf['pr-q4-9yr/dt'][:num,:,:,:3] # in 1.2*km
    qs = hf['pr-q4-9yr/dt'][:num,:,:,3:4]
    ev_params = hf['pr-q3-9yr/ev_params'][:num]
    
times_reg = times_flat + times_diff
mask_qs = tf.where( tf.math.logical_and(mask>0, qs>0), 1., 0. )
xs = tf.expand_dims( tf.linspace( 0., 5*1.2, 6 ), axis=-1 )
xs = tf.repeat(xs, 6, axis=1)
ys = tf.transpose( xs )
coords = tf.expand_dims( tf.stack( (xs,ys), axis=-1 ), axis=0 ) # in km
c = tf.constant(0.299792458, dtype=tf.float32) # in km\mks
# prepare matricies to solve
def make_matrix(coords, times_reg, mask):
    
    # matrix for dL/db
    hs = tf.ones((1,6,6,1)) # to make (x_1,x_2) -> (x_1,x_2,1), to account for b
    coords_ext = mask*tf.concat( (coords,hs), axis=-1 ) # (bs,6,6,3)
    matrix_b = tf.expand_dims( tf.math.reduce_sum( coords_ext, axis=(1,2) ), axis=-1 ) # (bs,6,6,3,1)
    times = mask*times_reg # (bs,6,6,1)
    rhs_b = tf.math.reduce_sum( times, axis=(1,2) ) # (bs,1)
    
    # matrix for dL/da
    coords_mul = tf.expand_dims( coords_ext, axis=-1) * tf.expand_dims( coords, axis=-2)
    matrix_a = tf.math.reduce_sum( coords_mul, axis=(1,2) ) # (bs,6,6,3,2)
    ts_mul = times * coords
    rhs_a = tf.math.reduce_sum( ts_mul, axis=(1,2) )
    
    matrix = tf.concat((matrix_b,matrix_a), axis=-1)
    rhs = tf.expand_dims( tf.concat((rhs_b,rhs_a), axis=-1), axis=-1 )
    
    return matrix, rhs
def lin_solve(matrix, rhs):
    sol_flat = tf.linalg.solve( matrix, rhs, adjoint=True )[:,:,0]
    t0 = sol_flat[:,2]
    # extract parameters
    n_z = 1/c/c - tf.math.reduce_sum(sol_flat[:,:2]*sol_flat[:,:2], axis=1)
    n_z = tf.where( n_z>0, tf.math.sqrt( n_z ), 0. ) # some configurations yield front moving faster than light
    print('n_z',n_z)
    theta_flat = tf.math.acos( n_z*c )
    phi_flat = tf.math.atan2( sol_flat[:,1], sol_flat[:,0] )
    
    return theta_flat, phi_flat, t0
matrix, rhs = make_matrix(coords, times_reg, mask)
sol_flat=lin_solve(matrix, rhs)
theta, phi, t0 = lin_solve(matrix, rhs) # (bs,)
phi/3.14*180


n_z tf.Tensor(
[2.3231223 2.7067738 2.220696  2.8793616 2.6138859 2.9728527 2.757923
 3.095263  3.0605483 3.2407243 3.1354082 2.691969  2.747927  2.989241
 3.040311  3.1288347 2.8474948 2.7300937 2.5441003 2.5787976 2.7840703
 2.898237  2.7465289 3.215001  3.1926513 2.8619928 3.0027409 3.0044591
 3.2113404 2.932823  2.1890938 2.8146274 3.1965425 2.7622495 2.626794
 2.995249  2.6168292 2.685593  2.8662877 2.935793  2.9724588 2.4492443
 3.0389907 3.216916  3.2808788 2.893057  2.5901816 3.226288  2.96118
 2.836399  0.        0.        3.1651006 3.0666704 2.34177   2.9919312
 2.7546403 2.7149653 3.0208242 2.9398496 2.6779318 2.7745664 3.295566
 2.8490405], shape=(64,), dtype=float32)
n_z tf.Tensor(
[2.3231223 2.7067738 2.220696  2.8793616 2.6138859 2.9728527 2.757923
 3.095263  3.0605483 3.2407243 3.1354082 2.691969  2.747927  2.989241
 3.040311  3.1288347 2.8474948 2.7300937 2.5441003 2.5787976 2.7840703
 2.898237  2.7465289 3.215001  3.1926513 2.8619928 3.0027409 3.0044591
 3.2113404 2.9

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ -45.090675 ,   59.493122 , -151.83202  ,  -66.795746 ,
        -40.908295 ,  109.61158  ,   25.507902 , -159.16087  ,
        -66.04494  , -117.72843  , -142.08736  ,  -64.07822  ,
         38.190773 ,  -72.61241  , -152.02588  ,  114.31395  ,
       -130.48126  ,  179.37828  ,  114.47934  ,   66.028435 ,
        -71.42547  ,  148.41838  ,  147.47336  ,  111.154564 ,
         99.09006  ,  111.39848  ,  -38.16081  , -173.6324   ,
        152.98344  ,   44.788975 ,  -45.31987  ,  -15.201984 ,
        138.35306  , -160.82552  ,   20.068356 ,  -95.30713  ,
         71.44137  , -154.94284  ,  107.48643  ,   48.002686 ,
          4.0393744,  -55.981457 ,  133.64186  , -174.19273  ,
         71.1309   ,  130.7888   ,  116.42755  ,  -37.481594 ,
         11.4989195,   85.55882  ,    1.0422605,  138.42151  ,
         57.84859  , -141.22237  ,   13.51256  ,  -13.543718 ,
       -105.87633  ,  178.18668  , -165.66962  ,  -29.445803 ,
        

In [169]:
matrix[:2]

<tf.Tensor: shape=(2, 3, 3), dtype=float32, numpy=
array([[[ 26.400002, 109.44    ,  90.72    ],
        [ 25.2     ,  90.72    ,  96.48    ],
        [  7.      ,  26.400002,  25.2     ]],

       [[ 27.599998, 113.76001 ,  99.36001 ],
        [ 25.2     ,  99.36001 ,  96.48    ],
        [  7.      ,  27.599998,  25.2     ]]], dtype=float32)>

In [167]:
sol[:2]

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[-1.6945673 ],
        [ 1.690574  ],
        [-1.0818663 ]],

       [[ 1.6789589 ],
        [ 0.99045706],
        [-9.411484  ]]], dtype=float32)>

In [168]:
sol_flat[:2]

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.690568 , -1.6945748, -1.0818156],
       [ 0.9904436,  1.678957 , -9.411423 ]], dtype=float32)>

In [81]:
t0_ext = t0[:,tf.newaxis,tf.newaxis]
nx = 1/c*tf.math.sin(theta)[:,tf.newaxis,tf.newaxis] * tf.math.cos(phi)[:,tf.newaxis,tf.newaxis]
ny = 1/c*tf.math.sin(theta)[:,tf.newaxis,tf.newaxis] * tf.math.sin(phi)[:,tf.newaxis,tf.newaxis]
a = tf.concat((nx,ny), axis=-1)
a = tf.expand_dims( a, axis=1 )

t_rec = t0_ext + tf.math.reduce_sum( a*coords, axis=-1 )
t_rec = t_rec*mask[:,:,:,0]
times_flat[:,:,:,0]*mask[:,:,:,0]

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -3.593     ,
         -5.576     ,  0.        ],
        [ 0.        ,  0.        ,  0.325     , -1.714     ,
         -3.748     ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -0.125     ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  3.8920002 ,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -1.665     ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -2.3170002 , -0.62      ,
          1.778     ,  0.        ],
        [ 0.        ,  0.   